In [11]:
# Add the current directory to Python path to find the brain package
import sys
from pathlib import Path

# Add the directory containing this notebook to Python path
# This allows importing the 'brain' package that's in the same directory
notebook_dir = Path(globals()["__vsc_ipynb_file__"]).parent.resolve()
if str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

# Import from the structured package
from brain import (
    BrainAgent,
    MultiModalEncoder,
    MultiModalDummyEnv,
    build_and_run_demo
)
import numpy as np

# Run the demo
if __name__ == "__main__":
    build_and_run_demo()


Global steps: 500
Memory size: 500
Drives (curiosity, competence): 0.07339804109036294 5.8149716295016006e-05
